# 단어 임베딩 - Text Embedding

## gensim을 사용한 임베딩 공간 탐색

In [1]:
# 이 코드는 tet8 데이터셋에서 Word2Vec 모델을 훈련시키고 이진 파일로 저장한다.
# Word2Vec 모델에는 많은 매개변수가 있지만 여기서는 기본 설정 값을 사용한다.
# 이 경우 창 크기가 5인(windows=5) CBOW 모델(sg=0)을 훈련하고 100차원 임베딩(size=100)을 생성한다.
import gensim.downloader as api
from gensim.models import Word2Vec

info = api.info("text8")
assert(len(info) > 0)

dataset = api.load("text8")
model = Word2Vec(dataset)

model.save("data/text8-word2vec.bin")

In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load('data/text8-word2vec.bin')
word_vectors = model.wv

- 어휘의 처음 몇 개 단어를 살펴보고 특정 단어가 있는지 살펴봄

In [ ]:
words = word_vectors.vocab.keys()
print([x for i, x in enumerate(words) if i < 10])
assert("king" in words)

- 다음과 같이 특정단어('king')와 유사한 단어를 검색할 수 있다.

In [ ]:
def print_most_similar(word_conf_pairs, k):
    for i, (word, conf) in enumerate(word_conf_pairs):
        print("{:.3f} {:s}".format(conf, word))
        if i >= k-1:
            break
    if k < len(word_conf_pairs):
        print("...")

print_most_similar(word_vectors.most_similar("king"), 5)

- ex) 프랑스의 수도는 파리다.
- ex) 독일의 수도는 베를린이다.
- 임베딩 공간에서 파리와 프랑스 사이의 거리가 베를린과 독일 사이의 거리와 같아야 한다.
- 다시 말해 프랑스 - 파리 + 베를린 = 독일 이어야 한다.
- cf) 단어 앞의 부동소수점 점수는 유사성 척도이다.

In [ ]:
print_most_similar(word_vectors.most_similar(
positive=['france', 'berlin'], negative=['paris']), 1)

- 좀 더 나은 유사도 척도도 gensim API에 구현돼 있다.
- 이 유사도값은 코사인 유사도다.

In [ ]:
print_most_similar(word_vectors.most_similar_cosmul(
positive=['france', 'berlin'], negative=['paris']), 1)

- gensim은 doesnt_match()라는 함수도 제공하는데, 단어의 목록 중 이상한 항목을 탐지하는데 사용할 수 있다.

In [ ]:
print(word_vectors.doesnt_match(['hindus', 'parsis', 'singapore', 'christians']))

# 결과는 예상대로 싱가폴, 종교들을 나열한 목록 중 싱가폴만이 유일한 국가명이기 때문이다.

In [ ]:
# 두 단어 사이의 유사성을 계산해주는 코드, 여기서는 관련된 단어 사이의 거리가
# 관련 없는 단어 사이의 거리보다 더 짧다는 것을 보여준다.
for word in ['woman','dog','whale','tree']:
    print('similarity({:s}, {:s}) = {:.3f}'.format(
    'man', word, word_vectors.similarity('man', word)))

- similar_by_word() 함수는 기능적으로는 similar() 함수와 같지만,<br>
  후자의 경우 기본적으로 비교하기 전에 벡터가 정규화된다.<br>
  또한 관련된 similar_by_vector() 함수가 있는데, 벡터를 입력으로 지정해<br>
  유사한 단어를 찾을 수 있다.
- singapore과 유사한 단어를 찾아봄

In [ ]:
print(print_most_similar(
word_vectors.similar_by_word('singapore'), 5))

# 최소한 지리적 관점에서는 zambia를 제외하고 모든 대답이 옳은 것처럼 보인다.
# 피부색의 관점에서는 모두 유사할수도(?)

- distance() 함수를 사용하면 임베딩 공간에서 두 단어 사이의 거리도 계산할 수 있다.
- 이 값은 사실 단순히 1 - similarity() 와 같다.

In [ ]:
print("distance(singapore, malaysia) = {:.3f}".format(
word_vectors.distance('singapore', 'malaysia')))

- 어휘 단어 벡터는 다음에서 보듯 word_vectors 객체를 직접 살펴보거나<br>
  word_vec() 래퍼를 사용해 찾아볼 수 있다.

In [ ]:
vec_song = word_vectors['song']
print("\n# output vector obtained directly, shape:", vec_song.shape)
vec_song_2 = word_vectors.word_vec('song', use_norm=True)
print("# output vector obtained using word_vec, shape:", vec_song_2.shape)

## 워드 임베딩을 사용한 스팸 탐지

In [ ]:
import argparse
import gensim.downloader as api
import numpy as np
import os
import shutil
import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix

- 다음 코드는 파일을 다운로드하고 파싱해 SMS 메시지 목록과 해당 레이블을 생성한다.

In [ ]:
def download_and_read(url):
    local_file = url.split('/')[-1]
    p = tf.keras.utils.get_file(local_file, url, extract=True, cache_dir='.')
    labels, texts = [],[]
    local_file = os.path.join('datasets', 'SMSSpamCollection')
    with open(local_file, 'r') as fin:
        for line in fin:
            label, text = line.strip().split('\t')
            labels.append(1 if label == 'spam' else 0)
            texts.append(text)
    return texts, labels

DATASET_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
texts, labels = download_and_read(DATASET_URL)

#### 데이터를 사용하기 위한 준비
- SMS 텍스트는 정수 시퀀스로 망에 공급돼야 한다.
- 여기서는 케라스 토크나이저를 사용해 각 SMS 텍스트를 단어 시퀀스로 변환한 후<br>
  토크나이저의 fit_on_texts() 메소드를 사용해 어휘를 생성한다.<br>
  그런 다음 texts_to_sequence()를 사용해 SMS 메세지를 정수 시퀀스로 변환한다.<br>
  마지막으로 망은 고정된 길이의 정수 시퀀스와만 작동되므로 pad_sequences() 함수를<br>
  호출해 더 짧은 SMS 메세지들은 0으로 채운다.

In [ ]:
# 토큰화하고 텍스트 채우기
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)
text_sequences = tokenizer.texts_to_sequences(texts)
text_sequences = tf.keras.preprocessing.sequence.pad_sequences(text_sequences)
num_records = len(text_sequences)
max_seqlen = len(text_sequences[0])
print("{:d} sentences, max length: {:d}".format(num_records, max_seqlen))

- 레이블은 범주형 또는 원 핫 인코딩 형식으로 변환, 사용하려는 손실 함수(범주형 교차 엔트로피)<br>
  가 해당 형식의 레이블을 받아들이기 때문

In [ ]:
# 레이블
NUM_CLASSES = 2
cat_labels = tf.keras.utils.to_categorical(labels, num_classes=NUM_CLASSES)

- 앞서 정의한 tokenizer는 기본적으로 word_index 속성으로 작성된 어휘에 액세스할 수 있게<br>
  해주는데, 이는 기본적으로 어휘의 색인 위치에 대한 어휘 단어 사전이다.
- 인덱스를 입력하면 단어를 알려주는거 하나, 단어를 입력하면 인덱스를 출력하는거 하나<br>
  총 2개의 사전을 만듬

In [ ]:
# 어휘
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
word2idx['PAD'] = 0
idx2word[0] = 'PAD'
vocab_size = len(word2idx)
print('vocab size: {:d}'.format(vocab_size))

- 마지막으로 dataset 객체를 만든다. dataset객체를 사용하면 batch size와 같은 일부 속성을<br>
  선언적으로 설정할 수 있다. 여기서는 채워진 정수와 범주 레이블 시퀀스에서<br>
  데이터셋을 작성하고 데이터를 섞은 다음 이를 훈련, 검증, 테스트 집합으로 분할한다.<br>
  그리고 세 가지 데이터셋 각각에 대한 batch size를 설정한다.

In [ ]:
# 데이터셋
dataset = tf.data.Dataset.from_tensor_slices((text_sequences, cat_labels))
dataset = dataset.shuffle(10000)
test_size = num_records // 4  # 5574 / 4 의 정수부분
val_size = (num_records - test_size) // 10  # (5574-1393)/10 의 정수부분
test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

BATCH_SIZE = 128
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

#### 임베딩 행렬 구축
- gensim 툴킷은 다양한 훈련된 임베딩 모델에 접근할 수 있게 해주는데,
- Word2Vec: 두가지 형식으로 제공, 하나는 구글뉴스(30억개토큰,300만개 단어벡터)로 훈련됐고, 또 하나는 러시아 말뭉치들(word2vec-ruscorpora-300,word2vec-google-news-300)로 훈련됐다.
- GloVe: 두가지 형식으로 제공, 하나는 Gigawords 말뭉치(60억개토큰,40만개 단어벡터)로 훈련된 것으로, 50d,100d,200d,300d 벡터로 제공. 다른 하나는 트위터(27억개토큰,120만개 단어벡터)에서 훈련된 것으로 25d, 50d, 100d, 200d 벡터로 제공
- fastTest: Wikipedia 2017의 부분 단어 정보로 훈련된 100만 단어벡터, UMBC웹 말뭉치와 statmt.org 뉴스 데이터셋(160억개 토큰)이다.
- ConceptNet Numberbatch: ConceptNet 의미 신경망, PPDB(Para Phrase DataBase), Word2Vec의 앙상블을 사용하고 GloVe를 입력으로 한다. 600차원 벡터를 생성

- 이 예에서는 Gigaword 말뭉치로 훈련된 300차원 GloVe 임베딩을 선택
- 모델 크기를 작게 유지하고자 어휘에 존재하는 단어의 임베딩만을 고려

In [ ]:
def build_embedding_matrix(sequences, word2idx, embedding_dim, embedding_file):
    if os.path.exists(embedding_file):
        E = np.load(embedding_file)
    else:
        vocab_size = len(word2idx)
        E = np.zeros((vocab_size, embedding_dim))
        word_vectors = api.load(EMBEDDING_MODEL)
        for word, idx in word2idx.items():
            try:
                E[idx] = word_vectors.word_vec(word)
            except KeyError:  # 임베딩에 없는 단어
                pass
            np.save(embedding_file, E)
    return E

EMBEDDING_DIM = 300
DATA_DIR = 'data'
EMBEDDING_NUMPY_FILE = os.path.join(DATA_DIR, "E.npy")
EMBEDDING_MODEL = 'glove-wiki-gigaword-300'
# 임베딩 행렬
E = build_embedding_matrix(text_sequences, word2idx, EMBEDDING_DIM,
    EMBEDDING_NUMPY_FILE)
print('Embedding matrix:', E.shape)

- 임베딩 출력 형태(9010, 300)은 어휘의 9010개 토큰과 제3자 GloVe 임베딩의 300개 특징에 해당된다.

#### 스팸 분류기 정의
- 감정 분석을 위해 1차원 컨볼루션 신경망을 사용함
- 입력은 정수 시퀀스
- 첫 번째 계층은 임베딩계층, 각 입력정수를 크기가 embedding_dim(300)인 벡터로 변환
- @@ 실행모드에 따라, 즉 임베딩을 처음부터 학습시키는지, 전이학습을 하는지, 아니면 미세 조정을 하는지에 따라 신경망의 임베딩 계층은 약간 다르다.
- @@ 신경망을 무작위로 초기화된 임베딩 가중치로 시작하고(run_mode == "scratch") 훈련 중 가중치를 학습하려면 trainable = True로 설정, 전이학습의 경우(run_mode == "vectorizer")에는 임베딩 행렬 E에서 가중치를 설정하지만 trainalbe = False로 설정, 미세조정의 경우(run_mode == "finetuning")에는 임베딩 가중치를 외부 행렬 E에서 설정하고 계층도 trainable = True로 설정
- 임베딩의 출력은 컨볼루션 계층에 공급
- 타임 스텝이라고도 하는 고정된 크기 3토큰너비의 1차원 창(kernel_size=3)은 256개 랜덤필터(num_filters=256)에 대해 컨볼루션돼 256크기의 벡터를 생성
- 출력 모양은 (batch_size,time_steps,num_filters)이다.
- 단어 임베딩 컨볼루션 계층의 출력은 1차원 공간 드롭아웃 계층으로 전송
- @@ 이는 컨볼루션 계층에서 출력된 전체 특징 맵을 임의로 삭제함 이것은 과적합을 방지하는 정규화 기술이다.
- 그런 다음 전역 최대풀링 계층을 통해 전송
- 이 계층은 각 필터의 각 타임 스텝에서 최댓값을 취하고 (batch_size,num_filters) 형태의 벡터를 생성
- 드롭아웃 계층의 출력은 밀집 계층(Dense)으로 공급
- (batch_size,num_filters) 형태의 벡터를 (batch_size,num_classes)로 변환
- softmax 활성화함수는 각(스팸,햄)에 대한 점수를 확률 분포로 변환, 입력 SMS가 각각 스팸 또는 햄일 확률을 나타냄

In [ ]:
class SpamClassifierModel(tf.keras.Model):
    def __init__(self, vocab_sz, embed_sz, input_length, num_filters, kernel_sz, output_sz, run_mode, embedding_weights, **kwargs):
        super(SpamClassifierModel, self).__init__(**kwargs)
        if run_mode == 'scratch':
            self.embedding = tf.keras.layers.Embedding(vocab_sz,
                embed_sz,
                input_length=input_length,
                trainable=True)
        elif run_mode == 'vectorizer':
            self.embedding = tf.keras.layers.Embedding(vocab_sz,
                embed_sz,
                input_length=input_length,
                weights=[embedding_weights],
                trainable=False)
        else:
            self.embedding = tf.keras.layers.Embedding(vocab_sz,
                embed_sz, input_length=input_length,
                weights=[embedding_weights],
                trainable=True)
        self.conv = tf.keras.layers.Conv1D(filters=num_filters,
            kernel_size=kernel_sz,
            activation='relu')
        self.dropout = tf.keras.layers.SpatialDropout1D(0.2)
        self.pool = tf.keras.layers.GlobalMaxPooling1D()
        self.dense = tf.keras.layers.Dense(output_sz,
            activation='softmax')
        
    def call(self, x):
        x = self.embedding(x)
        x = self.conv(x)
        x = self.dropout(x)
        x = self.pool(x)
        x = self.dense(x)
        return x

'''parser = argparse.ArgumentParser()
parser.add_argument("--mode", help="run mode",
    choices=[
        "scratch",
        "vectorizer",
        "finetuning"
    ])
args = parser.parse_args()
run_mode = args.mode'''
### SystemExit: 2 Error ###


# 모델 정의
conv_num_filters = 256
conv_kernel_size = 3
model = SpamClassifierModel(
    vocab_size, EMBEDDING_DIM, max_seqlen,
    conv_num_filters, conv_kernel_size, NUM_CLASSES,
    'finetuning', E)
model.build(input_shape=(None, max_seqlen))


In [ ]:
model.summary()

- 마지막으로 범주형 교차 엔트로피 손실 함수와 Adam 최적화기를 사용해 모델을 컴파일한다.

In [ ]:
# 컴파일
model.compile(optimizer="adam", loss='categorical_crossentropy',
    metrics=['accuracy'])

#### 모델의 훈련과 평가
- 한 가지 주목할 점은 데이터셋이 다소 불균형하다는 것이다.
- 햄 인스턴스가 4827개인 것에 비해 스팸은 단 747개만 있다.
- 신경망이 단순히 항상 다부류(이 경우는 햄)로만 예측해도 무려 87%에 가까운 정확도를 달성할 수 있는 셈이다.
- 이 문제를 완화하고자 스팸 SMS의 오류가 햄 SMS의 오류보다 8배 더 중요하다는 것을 나타내려고 부류 가중치를 설정함
- 이는 CLASSES_WEIGHTS 변수에 표시되며, 추가 매개변수로 model.fit()을 호출할 때 전달됨
- 훈련은 3에폭만 훈련, 모델의 정확도와 혼동행렬을 출력

In [ ]:
label_0, label_1 = [],[]
for i in labels:
    if i == 0:
        label_0.append(i)
    else:
        label_1.append(i)

print('전체 SMS:',len(labels))
print('햄:',len(label_0))
print('스팸:',len(label_1))

In [ ]:
NUM_EPOCHS = 3
# 데이터 분포를 보면 햄은 4827, 스팸은 747(전체는 5574)로,
# 약 87%가 햄이고 스팸은 단 13%다. 따라서 비율에 따라
# 스팸(1) 아이템은 햄(0)보다 약 8배의 가중치를 준다.
CLASS_WEIGHTS = {0:1, 1:8}

# 모델 훈련
model.fit(train_dataset, epochs=NUM_EPOCHS,
    validation_data=val_dataset,
    class_weight=CLASS_WEIGHTS)

# 테스트 집합으로 평가
pred_labels, predictions = [],[]
for Xtest, Ytest in test_dataset:
    Ytest_ = model.predict_on_batch(Xtest)
    ytest = np.argmax(Ytest, axis=1)
    ytest_ = np.argmax(Ytest_, axis=1)
    pred_labels.extend(ytest.tolist())
    predictions.extend(ytest.tolist())

print("test accuracy: {:.3f}".format(accuracy_score(pred_labels,predictions)))
print("confusion matrix")
print(confusion_matrix(pred_labels,predictions))

- 데이터셋은 작고 모델은 매우 단순하다. 3에폭이라는 최소한의 훈련만으로도 매우 우수한 결과를 달성할 수 있었다.
- 세 경우 모두 신경망은 1094개의 햄 메세지와 186개의 스팸 메세지를 정확하게 예측해 만점을 받았다.

## 신경망 임베딩

- 신경망 임베딩 : 단어 임베딩을 단어가 아닌 설정에 적용하는것, 단어 임베딩은 유사한 문맥에서 등장하는 단어들은 서로 비슷한 의미를 갖는 경향이 있다는 분포 가설을 활용, 문맥이란 일반적으로 대상 단어 주위의 고정크기(단어 개수)의 창이다. 신경망 임베딩 아이디어도 매우 유사, 즉 비슷한 맥락에서 발생하는 개체들은 서로 밀접하게 관련되는 경향이 있다는것. 이런 방법은 일반적으로 상황에 의존한다.
- Item2vec : 어떤 한 사용자와 유사한 구매 이력이 있는 다른 사용자의 구매를 기반으로 사용자에게 상품을 추천한다는 아이디어이다. 웹 스토어의 항목을 단어로, 아이템셋(시간에 따른 사용자의 순차적 구매 항목)을 문장으로 사용하여 단어 문맥을 도출한다. 예를 들어 슈퍼마켓에서 쇼핑객에게 상품을 추천하는 문제를 생각해볼 때, 슈퍼마켓에서 5000개의 상품을 판매한다고 사정하면 각 상품은 크기가 5000인 희소 원핫 인코딩 벡터로 표현가능. Word2Vec과 유사하게 문맥 창을 적용해 스킵그램 모델을 훈련시키면 가능한 항목 쌍을 예측 가능. 학습된 임베딩 모델은 유사한 항목이 서로 가깝게 위치하도록 밀집 저차원 공간으로 항목을 매핑해 유사한 항목을 추천하는데 사용할 수 있다.
- node2vec : 그래프에서 노드의 특징을 학습하는 기법으로 제안됨. 그래프에서 다수의 고정 길이 랜덤워크를 실행함으로써 그래프 구조의 임베딩을 학습. 노드는 단어이고 랜덤워크는 문장이며 이를 통해 단어문맥이 도출된다.
- 자신만의 신경망 임베딩을 생성하고자 node2vec과 유사한 모델을 생성함
- 자세히 말하자면 NeurIPS 컨퍼런스에 제출된 논문들에 등장하는 단어들 사이의 동시 발생을 활용해 제시한 그래프 기반 임베딩의 조상 격인 DeepWalk에 기반을 둔 모델이다.
- 데이터셋은 11463 x 5812 행렬로 단어 개수를 나타내며, 행은 단어 열은 컨퍼런스 논문을 나타냄. 여기서는 논문 그래프는 만드는데, 두 논문 사이의 에지(edge)는 두 논문 모두에 등장하는 단어를 나타낸다. node2vec과 DeepWalk는 모두 무방향과 무가중치 그래프를 가정한다. 한 쌍의 논문 사이의 관계는 양방향이므로 무방향 그래프가 된다. 그러나 에지(edge)는 두 논문 사이의 단어 동시 발생 횟수에 따라 가중치를 갖고 있을 수도 있다. 여기선 0보다 큰 값을 가진 모든 경우에 단순히 유효한 비가중치 에지로 간주함

In [1]:
import gensim
import logging
import numpy as np
import os
import shutil
import tensorflow as tf

from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
level=logging.INFO)

In [2]:
DATA_DIR = './datasets'
UCI_DATA_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00371/NIPS_1987-2015.csv'

def download_and_read(url):
    local_file = url.split('/')[-1]
    p = tf.keras.utils.get_file(local_file, url, cache_dir='.')
    row_ids, col_ids, data = [],[],[]
    rid = 0
    f = open(p, 'r')
    for line in f:
        line = line.strip()
        if line.startswith("\"\","):
            # 헤더
            continue
        # 현재 행의 0이 아닌 원소를 계산
        counts = np.array([int(x) for x in line.split(',')[1:]])
        nz_col_ids = np.nonzero(counts)[0]
        nz_data = counts[nz_col_ids]
        nz_row_ids = np.repeat(rid, len(nz_col_ids))
        rid += 1
        # 빅 리스트에 데이터를 추가
        row_ids.extend(nz_row_ids.tolist())
        col_ids.extend(nz_col_ids.tolist())
        data.extend(nz_data.tolist())
    print("{:d} rows read, COMPLETE".format(rid))
    f.close()
    TD = csr_matrix((
        np.array(data), (
            np.array(row_ids), np.array(col_ids)
        )
    ), shape=(rid, counts.shape[0]))
    return TD

In [ ]:
# 데이터를 읽고 용어-문서 행렬로 변환
TD = download_and_read(UCI_DATA_URL)
# 무방향 무가중치 에지 행렬 계산
E = TD.T * TD
# 이진화
E[E > 0] = 1
print(E.shape)

- 다음 코드는 램덤 워크를 수정하고 그 경로를 RANDOM_WALKS_FILE에서 지정한 파일에 저장한다. 이 과정은 매우 느리다.

In [3]:
NUM_WALKS_PER_VERTEX = 32
MAX_PATH_LENGTH = 40
RESTART_PROB = 0.15

RANDOM_WALKS_FILE = os.path.join(DATA_DIR, 'random-walks.txt')

def construct_random_walks(E, n, alpha, l, ofile):
    if os.path.exists(ofile):
        print('random walks generated already, skipping')
        return
    
    f = open(ofile, 'w')
    for i in range(E.shape[0]):  # 각 꼭짓점마다
        if i % 100 == 0:
            print("{:d} random walks generated from {:d} vertices".format(n * i, i))
        if i <= 3273:
            continue
        for j in range(n):       # n 랜던 워크 구축
            curr = i
            walk = [curr]
            target_nodes = np.nonzero(E[curr])[1]
            for k in range(l):   # each of max length l
                # 시작할 것인가?
                if np.random.random() < alpha and len(walk) > 5:
                    break
                # 외향 에지를 하나 골라서 워크에 첨가
                try:
                    curr = np.random.choice(target_nodes)
                    walk.append(curr)
                    target_nodes = np.nonzero(E[curr])[1]
                except ValueError:
                    continue
            f.write("{:s}\n".format(" ".join([str(x) for x in walk])))
    
    print("{:d} random walks generated from {:d} vertices, COMPLETE".format(n * i, i))
    f.close()

In [ ]:
# 랜덤 워크 구성(주의 : 상당히 시간이 걸리는 프로세스!)
#construct_random_walks(E, NUM_WALKS_PER_VERTEX, RESTART_PROB, MAX_PATH_LENGTH, RANDOM_WALKS_FILE)

random-walks.txt 의 몇 줄을 작성해보면<br>
0 1405 4845 754 4391 3524 4282 2357 3922 1667<br>
0 1341 456 495 1647 4200 5379 473 2311<br>
0 3422 3455 118 4527 2304 772 3659 2852 4515 5135 3439 1273
- 이는 단어들의 어휘가 그래프의 모든 노드id인 언어로 작성된 문장처럼 상상할 수 있다.
- 단어 임베딩이 언어의 구조를 이용해 단어의 분포 표현을 생성한다는 것을 바탕으로 DeepWalk나 node2vec과 같은 그래프 임베딩은 램덤 워크로 생성된 문장에서 동일한 기능을 수행한다. 이러한 임베딩은 다음에서 보는 것처럼 직접적인 이웃을 넘어서서 그래프의 노드 간 유사성까지 포착할 수 있다.

In [4]:
MODEL_DIR = './model/neurips_papers_n2v'
W2V_MODEL_FILE = os.path.join(MODEL_DIR, 'w2v-neurips-papers.model')

class Documents(object):
    def __init__(self, input_file):
        self.input_file = input_file

    def __iter__(self):
        with open(self.input_file, 'r') as f:
            for i, line in enumerate(f):
                if i % 1000 == 0:
                    if i % 1000 == 0:
                        logging.info("{:d} random walks extracted".format(i))
                yield line.strip().split()
    
def train_word2vec_model(random_walks_file, model_file):
    if os.path.exists(model_file):
        print('Model file {:s} already present, skipping training'.format(model_file))
        return
    docs = Documents(random_walks_file)
    model = gensim.models.Word2Vec(docs,
        size=128,    # 임베딩 벡터 크기
        window=10,   # 윈도우 크기(고정된 창)
        sg=1,        # 스킵그램 모델
        min_count=2,
        workers=4
    )
    model.train(
        docs,
        total_examples=model.corpus_count,
        epochs=50)
    model.save(model_file)

In [5]:
# 모델 훈련
train_word2vec_model(RANDOM_WALKS_FILE, W2V_MODEL_FILE)

Model file ./model/neurips_papers_n2v/w2v-neurips-papers.model already present, skipping training


- 결과 DeepWalk 모델은 바로 Word2Vec 모델이므로 단어 맥락에서 Word2Vec 으로 할 수 있는 모든 것은 이 모델에서는 꼭짓점의 맥락에서 할 수 있다. 이 모델을 사용해 문서 사이의 유사점을 참아봄

In [18]:
def evaluate_model(td_matrix, model_file, source_id):
    model = gensim.models.Word2Vec.load(model_file).wv
    most_similar = model.most_similar(str(source_id))
    scores = [x[1] for x in most_similar]
    target_ids = [x[0] for x in most_similar]
    # 소스의 각 타깃 사이에 상위 10개 코사인 유사성 비교
    X = np.repeat(td_matrix[source_id].todense(), 10, axis=0)
    Y = td_matrix[target_ids].todense()
    cosims = [cosine_similarity(X[i], Y[i])[0, 0] for i in range(10)]
    for i in range(10):
        print("{:d} {:s} {:.3f}".format(
            source_id, target_ids[i], cosims[i], scores[i]))

In [19]:
source_id = np.random.choice(E.shape[0])
evaluate_model(TD, W2V_MODEL_FILE, source_id)

2020-06-26 15:45:38,811 : INFO : loading Word2Vec object from ./model/neurips_papers_n2v/w2v-neurips-papers.model
2020-06-26 15:45:38,869 : INFO : loading wv recursively from ./model/neurips_papers_n2v/w2v-neurips-papers.model.wv.* with mmap=None
2020-06-26 15:45:38,870 : INFO : setting ignored attribute vectors_norm to None
2020-06-26 15:45:38,870 : INFO : loading vocabulary recursively from ./model/neurips_papers_n2v/w2v-neurips-papers.model.vocabulary.* with mmap=None
2020-06-26 15:45:38,871 : INFO : loading trainables recursively from ./model/neurips_papers_n2v/w2v-neurips-papers.model.trainables.* with mmap=None
2020-06-26 15:45:38,872 : INFO : setting ignored attribute cum_table to None
2020-06-26 15:45:38,872 : INFO : loaded ./model/neurips_papers_n2v/w2v-neurips-papers.model
2020-06-26 15:45:38,887 : INFO : precomputing L2-norms of word weight vectors


TypeError: cannot perform reduce with flexible type

## 동적 임베딩

In [4]:
# ELMo 동적 임베딩
import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/elmo/2"
tf.compat.v1.disable_eager_execution()

elmo = hub.Module(module_url, trainable=False)
embeddings = elmo([
    "i like green eggs and ham",
    "would you eat them in a box"],
    signature="default",
    as_dict=True
)['elmo']
print(embeddings.shape)

2020-06-27 20:39:35,018 : INFO : Using /var/folders/fs/qjp88bt91vn_8kbs8qkcc_z00000gn/T/tfhub_modules to cache modules.
2020-06-27 20:39:35,021 : INFO : Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
2020-06-27 20:39:57,245 : INFO : Downloading https://tfhub.dev/google/elmo/2: 30.35MB
2020-06-27 20:40:12,913 : INFO : Downloading https://tfhub.dev/google/elmo/2: 50.35MB
2020-06-27 20:40:30,714 : INFO : Downloading https://tfhub.dev/google/elmo/2: 60.35MB
2020-06-27 20:40:50,522 : INFO : Downloading https://tfhub.dev/google/elmo/2: 90.35MB
2020-06-27 20:41:17,602 : INFO : Downloading https://tfhub.dev/google/elmo/2: 130.35MB
2020-06-27 20:41:41,592 : INFO : Downloading https://tfhub.dev/google/elmo/2: 170.35MB
2020-06-27 20:41:59,873 : INFO : Downloading https://tfhub.dev/google/elmo/2: 190.35MB
2020-06-27 20:42:15,820 : INFO : Downloading https://tfhub.dev/google/elmo/2: 200.35MB
2020-06-27 20:42:38,888 : INFO : Downloading https://tfhub.dev/google/elmo/2: 230.35MB
2020-06

## 문장과 문단 임베딩

In [13]:
# Google Universal Sentence Encoder
import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
tf.compat.v1.disable_eager_execution()

model = hub.Module(module_url)
embeddings = model([
    "i like green eggs and ham",
    "would you eat them in a box"
])
with tf.compat.v1.Session() as sess:
    sess.run([
        tf.compat.v1.global_variables_initializer(),
        tf.compat.v1.tables_initializer()
    ])
    embeddings_value = sess.run(embeddings)

print(embeddings_value.shape)

2020-06-27 21:00:51,986 : INFO : Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
2020-06-27 21:01:09,134 : INFO : Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 18.00MB
2020-06-27 21:01:29,641 : INFO : Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 48.00MB
2020-06-27 21:01:47,686 : INFO : Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 78.00MB
2020-06-27 21:02:03,051 : INFO : Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 118.00MB
2020-06-27 21:02:21,101 : INFO : Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 158.00MB
2020-06-27 21:02:38,945 : INFO : Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 198.00MB
2020-06-27 21:02:57,685 : INFO : Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 248.00MB
2020-06-27 21:03:13,331 : INFO : Downloading https://tfhub.dev/google/universal-sentence-encoder/2: 278.00MB
2020-06-27 21:0